In [103]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")
from pymystem3 import Mystem

noise = stopwords.words('russian') + list(punctuation)
upnoise = [letter.upper() for letter in noise]
sum_noise = noise+upnoise
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import math

[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [163]:
df = pd.read_excel('youtube_ch_info.xlsx')
df = df.dropna()
df.index = np.arange(len(df))

In [164]:
df

,name,link,subs,request,views,date,desc
0,ЗАКОННАЯ НЕДВИЖИМОСТЬ СОЧИ ОСОКИН АЛЕКСЕЙ,https://www.youtube.com/channel/UCSjnoPNV8VK7U...,11800,дизайнерский ремонт квартир под ключ,995385,2016-11-27 00:00:00,Помогу Вам купить лучшую квартиру в Сочи! Раб...
1,Verona - Ремонт квартир и ремонт домов,https://www.youtube.com/channel/UCaCJNkb1nEt7G...,0,дизайнерский ремонт квартир под ключ,44,2020-11-28 00:00:00,"Ремонтно-строительная компания ""Верона-ремонт..."
2,Проект Групп,https://www.youtube.com/channel/UCKudV_r0_DkfS...,0,дизайнерский ремонт квартир под ключ,9092,2019-03-25 00:00:00,Компания «Проект Групп» ведёт свою профессион...
3,Forcemontage - Ремонт Квартир Ремонт домов,https://www.youtube.com/user/forcemontage,138000,дизайнерский ремонт квартир под ключ,18700880,2013-01-25 00:00:00,"Форс Монтаж - Ремонт квартир, ремонт ванных к..."
4,Жена строителя No1 Ремонт квартир в СПБ,https://www.youtube.com/channel/UCIxo8XOeJWJaG...,23900,дизайнерский ремонт квартир под ключ,2266446,2017-04-17 00:00:00,"Я Вика, а это канал о тяжелой жизни жены стро..."
...,...,...,...,...,...,...,...
3964,GOR STROY,https://www.youtube.com/channel/UCk-4OyDXzPLWX...,0,элитный ремонт квартир под ключ,1584,2016-04-02 00:00:00,Строительно отделочные работы от эконома до л...
3965,SanKraft Березовский Станислав,https://www.youtube.com/channel/UC01wSD2-S0m3q...,0,элитный ремонт квартир под ключ,151531,2013-12-14 00:00:00,"Инженерные решения, элитная отделка, Сантехни..."
3966,Александр Архипов,https://www.youtube.com/channel/UC6u5Kliw-Ll-x...,0,элитный ремонт квартир под ключ,28,2015-08-29 00:00:00,"Добрый день, меня зовут Архипов Александр и я..."
3967,GOR STROY,https://www.youtube.com/channel/UCv-Ks0G81aQJi...,0,элитный ремонт квартир под ключ,128,2019-02-27 00:00:00,Строительно отделочные работы от эконома до л...


In [14]:
req = pd.read_excel('smantika_remonty.xlsx',  sheet_name = None, header=0)

In [26]:
req['ремонт'].columns = ['name']
req['дизайн'].columns = ['name']
req['дизайн и ремонт'].columns = ['name']
req['дизайнер'].columns = ['name']

In [48]:

keys = pd.concat([req['ремонт'], req['дизайн'], req['дизайн и ремонт'], req['дизайнер']], axis=0, ignore_index=True)

In [134]:
keys = pd.read_excel('keys.xlsx', index_col = 0)

In [136]:
keys.columns = ['name', 'id']

In [143]:
srez = (keys['id'] == 1)
keys = keys[~srez]['name'].values

In [37]:

np.savetxt('keys.csv', keys, delimiter=',', fmt='%s')

In [42]:
keys[1].strip().split()

['какие', 'ремонты', 'делают', 'в', 'квартирах']

In [144]:
keywords = []
for k,ls in enumerate(keys):
    keywords.append(ls.strip().split())

In [145]:
keywords

[['ванна'],
 ['квартира'],
 ['ванна'],
 ['новостройка'],
 ['ремонт'],
 ['ванная'],
 ['кв'],
 ['стиль'],
 ['типовой'],
 ['прихожая'],
 ['вторичка'],
 ['интерьер'],
 ['квартира'],
 ['квартира'],
 ['дизайн'],
 ['квадратный'],
 ['ремонт'],
 ['капитальный'],
 ['кухня'],
 ['прихожая'],
 ['гостиная'],
 ['квартирв'],
 ['дом'],
 ['дизайн'],
 ['метр'],
 ['проект'],
 ['трехкомнатный'],
 ['стиль'],
 ['ремонт'],
 ['дизайнер'],
 ['студия'],
 ['однокомнатный'],
 ['двухкомнатный'],
 ['кухня'],
 ['м'],
 ['ванный'],
 ['ремонт'],
 ['дом'],
 ['гостиная'],
 ['дизайн'],
 ['светлый'],
 ['дизайнер'],
 ['интерьер'],
 ['бесплатный'],
 ['дизайн'],
 ['светлый'],
 ['проект'],
 ['новостройка'],
 ['дизайнер'],
 ['студия'],
 ['комнатный'],
 ['современный'],
 ['тон'],
 ['квартира'],
 ['ванная'],
 ['коридор'],
 ['балкон'],
 ['квартира'],
 ['новостройка'],
 ['москва'],
 ['комната'],
 ['материал'],
 ['дизайн'],
 ['однакомнатный']]

In [146]:
flat_list = [item for sublist in keywords for item in sublist]

In [147]:
final_keys = [word for word in flat_list if word not in sum_noise]

In [148]:
final_keys = list(set(final_keys))

In [149]:
final_keys = mystem_analyzer.lemmatize(det.detokenize(final_keys))

In [166]:
for i in range(len(df)):
    try:
        if df['desc'][i]>0:
            print(i)
            df['desc'][i] = np.nan
    except TypeError:
        continue
df = df.dropna()
df.index = np.arange(len(df))

In [167]:
tw = TweetTokenizer()
det = TreebankWordDetokenizer()
for i in (range(len(df))):
    try:
        tokenized_example = (tw.tokenize(df["desc"][i]))
        filtered_example = [word for word in tokenized_example if not word in sum_noise]
        df["desc"][i] = det.detokenize(filtered_example)
    except:
        print(i)

mystem_analyzer = Mystem(entire_input=False)
for i in tqdm(range(len(df))):
    df["desc"][i] = mystem_analyzer.lemmatize(df['desc'][i])

/home/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
  0%|          | 0/3964 [00:00<?, ?it/s]/home/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
100%|██████████| 3964/3964 [00:05<00:00, 740.35it/s]


In [168]:
'ремонт' in df['desc'][1]

True

In [169]:
idx = []
for i in tqdm(range(len(df))):
    for keyword in final_keys:
        if keyword in df['desc'][i]:
            idx.append(df['name'][i])
            break
    

100%|██████████| 3964/3964 [00:00<00:00, 4842.84it/s]


In [ ]:
df1 = pd.read_

In [170]:
idx

['ЗАКОННАЯ НЕДВИЖИМОСТЬ СОЧИ ОСОКИН АЛЕКСЕЙ',
 'Verona - Ремонт квартир и ремонт домов',
 'Проект Групп',
 'Forcemontage - Ремонт Квартир Ремонт домов',
 'Жена строителя No1 Ремонт квартир в СПБ',
 'm2 development',
 'БюроРемонт Дизайн интерьера',
 '1 - Ремонт квартир в Севастополе. Братья Кравчук.',
 'UDOBNO MOSCOW',
 'Vip Construction',
 'Мастер-классы по ремонту квартир',
 'Ремонт квартир под ключ. Юрий Литвин',
 'Ремонт квартир в Москве',
 'Студия дизайна интерьера - ОлимпСтройСервис',
 'Дизайнерский ремонт под ключ - сервис ARXNADZOR',
 'IZZYSTROY',
 'pobedastroy',
 'Супер ремонт квартир в СПб',
 'RemontStyle',
 'orionstroy kiev',
 'Ремонт квартир в СПБ Ремонт квартиры в СПБ',
 'ПОД КЛЮЧ',
 'Rsk Stroy',
 'ГК Фундамент Дизайн интерьера и ремонт квартир',
 'Бородатый Прораб',
 'Lebren Комплексный ремонт квартир',
 'Дизайнерский ремонт Специнстрой',
 'СД РОССТРОЙ - дизайн и ремонт квартир',
 'Мастерсити',
 'ремонт квартир в спб Санкт-Петербург',
 'Рем Брандт Ремонт',
 'Profirem',
 'Р

In [174]:

clear = pd.DataFrame()
clear.append(df.loc[1])

,date,desc,link,name,request,subs,views
1,2020-11-28 00:00:00,"[ремонтно-строительный, компания, верона, ремо...",https://www.youtube.com/channel/UCaCJNkb1nEt7G...,Verona - Ремонт квартир и ремонт домов,дизайнерский ремонт квартир под ключ,0.0,44.0


In [175]:
clear

""


In [178]:
df = pd.read_excel('youtube_ch_info.xlsx')
clear = pd.DataFrame()
for i in tqdm(idx):
    for j in df['name']:
        if i==j:
            clear = clear.append(df[df['name']==j])
            break

100%|██████████| 3379/3379 [00:10<00:00, 336.93it/s]


In [182]:
clear = clear.drop_duplicates(subset=['name'])

In [183]:
clear.to_excel('filtered_youtube_info.xlsx')

In [ ]:
trash = ['современный', "технический", "бесплатно", "отзыв", "хороший", ""]

In [129]:
pd.DataFrame(final_keys).to_excel('keys.xlsx')

array(['ванна', 'квартира', 'ванна', 'новостройка', 'ремонт', 'ванная',
       'кв', 'стиль', 'типовой', 'прихожая', 'вторичка', 'интерьер',
       'квартира', 'квартира', 'дизайн', 'квадратный', 'ремонт',
       'капитальный', 'кухня', 'прихожая', 'гостиная', 'квартирв', 'дом',
       'дизайн', 'метр', 'проект', 'трехкомнатный', 'стиль', 'ремонт',
       'дизайнер', 'студия', 'однокомнатный', 'двухкомнатный', 'кухня',
       'м', 'ванный', 'ремонт', 'дом', 'гостиная', 'дизайн', 'светлый',
       'дизайнер', 'интерьер', 'бесплатный', 'дизайн', 'светлый',
       'проект', 'новостройка', 'дизайнер', 'студия', 'комнатный',
       'современный', 'тон', 'квартира', 'ванная', 'коридор', 'балкон',
       'квартира', 'новостройка', 'москва', 'комната', 'материал',
       'дизайн', 'однакомнатный'], dtype=object)